## 1. Introduction

<p>Version control repositories like CVS, Subversion or Git can be a real gold mine for software developers. They contain every change to the source code including the date (the "when"), the responsible developer (the "who"), as well as a little message that describes the intention (the "what") of a change.</p>
<p>In this notebook, we will analyze the evolution of a very famous open-source project &ndash; the Linux kernel. The Linux kernel is the heart of some Linux distributions like Debian, Ubuntu or CentOS. Our dataset at hand contains the history of kernel development of almost 13 years (early 2005 - late 2017). We get some insights into the work of the development efforts by </p>
<ul>
<li>identifying the TOP 10 contributors and</li>
<li>visualizing the commits over the years.</li>
</ul>

In [13]:
#Importing required pacakages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
print('[info]:  Success')

[info]:  Success


In [14]:
# Let's see the content of git_log_excerpt.csv
print(pd.read_csv('git_log_excerpt.csv'))

      1502382966#Linus Torvalds
0       1501368308#Max Gurtovoy
1        1501625560#James Smart
2        1501625559#James Smart
3       1500568442#Martin Wilck
4           1502273719#Xin Long
5    1502278684#Nikolay Borisov
6  1502238384#Girish Moodalbail
7   1502228709#Florian Fainelli
8     1502223836#Jon Paul Maloy


## 2. Reading in the dataset
<p>The dataset was created by using the command <code>git log --encoding=latin-1 --pretty="%at#%aN"</code> in late 2017. The <code>latin-1</code> encoded text output was saved in a header-less CSV file. In this file, each row is a commit entry with the following information:</p>
<ul>
<li><code>timestamp</code>: the time of the commit as a UNIX timestamp in seconds since 1970-01-01 00:00:00 (Git log placeholder "<code>%at</code>")</li>
<li><code>author</code>: the name of the author that performed the commit (Git log placeholder "<code>%aN</code>")</li>
</ul>
<p>The columns are separated by the number sign <code>#</code>. The complete dataset is in the <code>datasets/</code> directory. It is a <code>gz</code>-compressed csv file named <code>git_log.gz</code>.</p>

In [23]:
# Reading in the log file
git_log = pd.read_csv('/content/git_log.gz',sep='#',encoding='latin-1',header=None,names=['timestamp','author'], lineterminator='\t')

# Printing out the first 5 rows
git_log.head()

,timestamp,author
ï¿½kd,],ï¿½ï¿½ï¿½ï¿½Èï¿½ï¿½ï¿½ ï¿½ï¿½ï¿½}ï¿½ï¿½gï¿...
ï¿½\nï¿½{@ï¿½ï¿½ï¿½ÌYÕVï¿½L9ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½]r]7qï¿½ï¿½ï¿½EZ!ï¿½,NaN,NaN
"ï¿½ï¿½USï¿½*ï¿½ï¿½Vï¿½ï¿½9Cdï¿½2ï¿½Hï¿½ï¿½|ï¿½ï¿½ï¿½ï¿½ï¿½_ï¿½\t^yï¿½ï¿½M/ï¿½ï¿½ï¿½&6ï¿½c~ï¿½m7%ï¿½ï¿½xï¿½ï¿½ï¿½ï¿½uï¿½|ï¿½7ï¿½ï¿½ï¿½{ï¿½ziï¿½ï¿½7ï¿½}ï¿½ï¿½iwï¿½Eï¿½x&ï¿½ï¿½W_oï¿½&ï¿½ï¿½ï¿½ï¿½ï¿½fÐLfï¿½ï¿½Vï¿½Ê«Ulï¿½ï¿½vï¿½ï¿½ ï¿½cï¿½ï¿½ ï¿½ï¿½ï¿½Uï¿½ï¿½ï¿½ï¿½zï¿½ï¿½mï¿½zÐ Oï¿½Ôï¿½ï¿½)>ï¿½ï¿½ï¿½.ï¿½ï¿½ï¿½ï¿½]G(oï¿½qï¿½;<ï¿½66ï¿½ï¿½qhï¿½\t-ï¿½Ktï¿½Ï¢ï¿½gRï¿½,ï¿½ï¿½ï¿½ï¿½/ï¿½ë¦¼ï¿½ï¿½Lï¿½Ø® Zï¿½ï¿½34ï¿½\ï¿½ï¿½]ï¿½Ùï¿½ï¿½$ï¿½ï¿½ï¿½ï¿½gï¿½ï¿½Hï¿½ï¿½Iï¿½\Yï¿½ï¿½Ïï¿½ï¿½9ï¿½ï¿½Mß4ï¿½I^ï¿½ï¿½ï¿½ï¿½ï¿½-ï¿½)ï¿½ï¿½ï¿½hoN\nï¿½8ï¿½ï¿½2/ï¿½ï¿½Pï¿½r.ï¿½ï¿½ï¿½!e3cOW ï¿½ï¿½Z8ï¿½~ï¿½1ï¿½ï¿½ï¿½ï¿½Clï¿½W bpï¿½ï¿½rï¿½Fï¿½\(/ï¿½ï¿½apNï¿½iTï¿½ï¿½ï¿½)ï¿½ï¿½Ígqï¿½ï¿½Rï¿½*ï¿½ ï¿½Mï¿½ï¿½ï¿½ÍMï¿½ï¿½Tï¿½",NaN,NaN
ï¿½ï¿½_ï¿½ï¿½1/ï¿½ï¿½H-ï¿½ï¿½ï¿½fï¿½ï¿½9I>B0ï¿½7Rï¿½ï¿½GÝyMï¿½ï¿½5ï¿½ï¿½$lFï¿½oï¿½ï¿½ï¿½ï¿½icï¿½ï¿½_Uï¿½q[uï¿½Ê¤6<eCð£ï¿½|ï¿½ ï¿½ï¿½ï¿½p\r\n|Ê«ï¿½1ï¿½[J9ï¿½ï¿½K,ZHï¿½ï¿½ï¿½vï¿½ï¿½ï¿½ï¿½ï¿½ï¿½dJï¿½{>ï¿½ï¿½ï...,ï¿½ï¿½ï¿½>| =ï¿½^ï¿½ï¿½ï¿½ï¿½Xï¿½ï¿½!Bæ§¸p...
"x6×ï¿½ï¿½%.ï¿½wï¿½ï¿½ n=Bï¿½Bï¿½ï¿½\Oï¿½ï¿½ï¿½ï¿½)ï¿½ï¿½Qvï¿½ï¿½,'ï¿½@ ^ï¿½Ã°ï¿½:ï¿½Oï¿½Lzk\rï¿½Gï¿½ï¿½yÝÉ§r]ï¿½ï¿½`nBCK^Qï¿½ï¿½>*ï¿½ï¿½ ï¿½N@r&ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½8hï¿½ï¿½3ï¿½ï¿½qfï¿½lï¿½ï¿½+ï¿½Cï¿½ï¿½\nï¿½ï¿½7wÇï¿½Cï¿½ï¿½.ï¿½mï¿½ï¿½ï¿½ï¿½(U@",7ï¿½6ï¿½Åï¿½ï¿½ï¿½0Lï¿½ï¿½ 1ï¿½Mï¿½ï¿½ï¿½ï...,*ï¿½`ï¿½ï¿½yï¿½Yï¿½ï¿½ï¿½ï¿½ï¿½? d(Rï¿½ï¿½ï¿½...


## 3. Getting an overview
<p>The dataset contains the information about every single code contribution (a "commit") to the Linux kernel over the last 13 years. We'll first take a look at the number of authors and their commits to the repository.</p>

In [24]:
# calculating number of commits
a,b = git_log.shape
number_of_commits = a

# calculating number of authors
number_of_authors = len(git_log.author.dropna().unique())

# printing out the results
print("%s authors committed %s code changes." % (number_of_authors, number_of_commits))

4362 authors committed 16960 code changes.


## 4. Finding the TOP 10 contributors
<p>There are some very important people that changed the Linux kernel very often. To see if there are any bottlenecks, we take a look at the TOP 10 authors with the most commits.</p>

In [ ]:
git_log.author.value_counts()[:10].index

In [ ]:
# Identifying the top 10 authors
top_10_authors = git_log.author.value_counts()[:10]

# Listing contents of 'top_10_authors'
top_10_authors

## 5. Wrangling the data
<p>For our analysis, we want to visualize the contributions over time. For this, we use the information in the <code>timestamp</code> column to create a time series-based column.</p>

In [ ]:
# converting the timestamp column
git_log.timestamp = pd.to_datetime(git_log.timestamp,unit="s")
# summarizing the converted timestamp column
git_log.timestamp.describe()

## 6. Treating wrong timestamps
<p>As we can see from the results above, some contributors had their operating system's time incorrectly set when they committed to the repository. We'll clean up the <code>timestamp</code> column by dropping the rows with the incorrect timestamps.</p>

In [ ]:
git_log.timestamp.sort_values()[:5]

In [ ]:
# determining the first real commit timestamp
first_commit_timestamp = '2005-04-16 22:20:36'

# determining the last sensible commit timestamp
last_commit_timestamp = '2017-10-03 12:57:00'

# filtering out wrong timestamps
corrected_log = git_log[(git_log.timestamp >= first_commit_timestamp) & (git_log.timestamp <= last_commit_timestamp)]

# summarizing the corrected timestamp column
corrected_log.describe()

## 7. Grouping commits per year
<p>To find out how the development activity has increased over time, we'll group the commits by year and count them up.</p>

In [ ]:
# Counting the no. commits per year
commits_per_year = corrected_log.groupby(pd.Grouper(key='timestamp',freq='AS')).count()

# Listing the first rows
commits_per_year.head(5)

## 8. Visualizing the history of Linux
<p>Finally, we'll make a plot out of these counts to better see how the development effort on Linux has increased over the the last few years. </p>

In [ ]:
# Setting up plotting in Jupyter notebooks
%matplotlib inline
# plot the data
commits_per_year.plot.line()
plt.figure(figsize=[10,12])
plt.title('No. of commits over years on Linux')
plt.ylabel('No. of Commits')
plt.yticks([20000,30000,40000,50000,60000,70000],['20K','30K','40K','50K','60K','70K'])
plt.xlabel('Year')
plt.show()

## 9.  Conclusion
<p>Thanks to the solid foundation and caretaking of Linux Torvalds, many other developers are now able to contribute to the Linux kernel as well. There is no decrease of development activity at sight!
The most number of commits were placed in year 2016</p>